In [4]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_109496/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [2]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_2_24,0.998322,0.721527,0.995783,0.998178,0.996878,0.000824,0.165314,0.002095,0.000762,0.001428,0.013608,0.028709,0.997004,0.028940,124.202148,280.197653,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
1,model_1_2_23,0.998231,0.721743,0.995640,0.998180,0.996802,0.000869,0.165185,0.002166,0.000761,0.001463,0.014056,0.029482,0.996841,0.029718,124.095954,280.091458,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
2,model_1_2_22,0.998033,0.722240,0.995430,0.998206,0.996699,0.000967,0.164891,0.002270,0.000750,0.001510,0.014975,0.031089,0.996487,0.031339,123.883623,279.879128,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
3,model_1_2_21,0.997900,0.722116,0.995263,0.998148,0.996582,0.001032,0.164964,0.002353,0.000774,0.001564,0.015809,0.032124,0.996249,0.032382,123.752584,279.748089,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
4,model_1_2_20,0.997724,0.722627,0.994977,0.998125,0.996416,0.001118,0.164661,0.002495,0.000784,0.001640,0.016525,0.033443,0.995935,0.033711,123.591677,279.587181,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,model_1_0_4,0.308583,0.391346,-0.269524,0.175938,0.061308,0.339700,0.361324,0.326829,0.448087,0.387458,0.315648,0.582838,-0.234673,0.587519,112.159387,268.154892,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
71,model_1_0_2,0.235696,0.168031,0.025577,0.241434,0.196477,0.375510,0.493893,0.250858,0.412473,0.331665,0.382279,0.612788,-0.364828,0.617710,111.958942,267.954447,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
72,model_1_0_1,0.200789,0.113781,-0.061278,0.253349,0.177239,0.392660,0.526098,0.273218,0.405994,0.339606,0.396419,0.626626,-0.427163,0.631659,111.869623,267.865128,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
73,model_1_0_3,0.176993,0.316143,-0.585333,-0.001650,-0.154150,0.404351,0.405967,0.408132,0.544651,0.476391,0.343426,0.635886,-0.469656,0.640994,111.810943,267.806448,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
